<center>
<h1>Working Examples</h1>
<br>
</center> 

<h3>1) Connect to a SQL Database</h3> <br>
We will use an in-memory only SQLite database. An in-memory SQLite database does not have a physical file and exists only in the memory. 

In [1]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

#Setting echo to TRUE enables logging.
#The return value of create_engine() is an instance of Engine, and it represents the interface to the database.

<h3>2) Declare a mapping</h3>

We have 2 steps to perform:
1. Describe the database table
2. Create our own classes that will be mapped to these tables. 

We can combine these 2 steps using a **declarative base class**. We will create the base class, and see how to create a class called **Customer** that will be mapped to the **Customer** table in the database. At the minimum, we have to specify the table name and the primary key column in order to create a class mapped onto some table.

In [2]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String
class Customer(Base):
    __tablename__ = 'Customer'
    id = Column(Integer, primary_key=True)
    firstname = Column(String)
    lastname = Column(String)

    def __repr__(self):
        return "<Customer(firstname='%s', lastname='%s')>" % (self.firstname, self.lastname)

<h3>3) Create a Schema</h3>

We have created a class **Customer** in which we have defined the information about the table (this is called the **metadata**). We will now create the actual table in the database. 

We can use MetaData to issue CREATE TABLE statements to the database for all tables that don’t yet exist. Below, we call the MetaData.create_all() method, passing in our Engine as a source of database connectivity.

In [3]:
Base.metadata.create_all(engine)
#the logging data below shows us that the existance of the table users (PRAGMA table_info("users")) was first checked before the 
#create command was issued.

2017-02-17 17:37:42,001 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-02-17 17:37:42,010 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:42,030 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-02-17 17:37:42,032 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:42,032 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Customer")
2017-02-17 17:37:42,032 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:42,048 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Customer" (
	id INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	PRIMARY KEY (id)
)


2017-02-17 17:37:42,048 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:42,063 INFO sqlalchemy.engine.base.Engine COMMIT


<h3>4) Instantiating the User class</h3>

Now that we have created the table, let us see how to create objects of the table. We will eventually use these objects in an INSERT statement.

In [4]:
customer1 = Customer(firstname = "Jane", lastname = "Doe") #note that we are NOT assigning the primary key

In [5]:
#we can access each attribute of the object as follows:
customer1.firstname

'Jane'

In [6]:
customer1.lastname

'Doe'

In [7]:
customer1.id #note that this attribute is empty

In [8]:
str(customer1.id)

'None'

<h3>5) Creating a session <br></h3>
We will now create a session. This variable will be used to talk to the database.

In [9]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In case you did not set up the Engine as previously mentioned, you can create the session as: <br><br>
<center> Session = sessionmaker() <br><br> </center>
and then configure it with the engine once it becomes available, as below: <br><br>
<center> Session.configure(bind=engine)</center>

The sessionmaker() creates a Session class. Whenever you want to "talk" to the database, you have to **instantiate** the class

In [10]:
session = Session()

<br><h3> 6) Adding and Updating Objects </h3>
<br><B>add()</B> in used to insert values into the table. **query()** is used for selection and updation operations.

In [11]:
session.add(customer1)

This will insert the user1 object that we previously created. However, the database will not be updated until a **flush** occurs. Flush occurs when we query the database for "Jane Doe". 

In [12]:
retrievedCustomer = session.query(Customer)
retrievedCustomer.first()

2017-02-17 17:37:43,238 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-17 17:37:43,241 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:43,249 INFO sqlalchemy.engine.base.Engine ('Jane', 'Doe')
2017-02-17 17:37:43,258 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer"
 LIMIT ? OFFSET ?
2017-02-17 17:37:43,262 INFO sqlalchemy.engine.base.Engine (1, 0)


<Customer(firstname='Jane', lastname='Doe')>

The logging clearly shows how the INSERT statement is executed **before** the SELECT statement. <br><br><br>
To insert multiple rows into the table, add_all() is used as follows:

In [13]:
session.add_all([
     Customer(firstname = 'Abriella', lastname = 'Peters'),
     Customer(firstname = 'Monica', lastname = 'Geller'),
     Customer(firstname = 'Chandler', lastname = 'Bing')])

Say we want to change the first name for customer1.

In [14]:
customer1.firstname = "Jessica"

As mentioned before, the data hasn't been committed to the database. If you wish to see the pending transactions:<br> **session.dirty** will tell you all the modified records that are still pending <br>
**session.new** will tell you all the new records that haven't been inserted

In [15]:
session.dirty

IdentitySet([<Customer(firstname='Jessica', lastname='Doe')>])

In [16]:
session.new

IdentitySet([<Customer(firstname='Monica', lastname='Geller')>, <Customer(firstname='Abriella', lastname='Peters')>, <Customer(firstname='Chandler', lastname='Bing')>])

Let's commit the pending transactions using session.commit

In [17]:
session.commit()

2017-02-17 17:37:43,992 INFO sqlalchemy.engine.base.Engine UPDATE "Customer" SET firstname=? WHERE "Customer".id = ?
2017-02-17 17:37:43,997 INFO sqlalchemy.engine.base.Engine ('Jessica', 1)
2017-02-17 17:37:44,003 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:44,008 INFO sqlalchemy.engine.base.Engine ('Abriella', 'Peters')
2017-02-17 17:37:44,016 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:44,020 INFO sqlalchemy.engine.base.Engine ('Monica', 'Geller')
2017-02-17 17:37:44,029 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:44,034 INFO sqlalchemy.engine.base.Engine ('Chandler', 'Bing')
2017-02-17 17:37:44,041 INFO sqlalchemy.engine.base.Engine COMMIT


In [18]:
session.new #empty set

IdentitySet([])

In [19]:
session.dirty #empty set

IdentitySet([])

It is also possible to **rollback** any changes made to the database. <br><br>
Let's change the name of customer1 to Emily from Jane. Also, let's create a dummy user just to see how rollback works.

In [20]:
customer1.firstname = "Emily"

In [21]:
dummyUser = Customer(firstname = "Joey", lastname = "Tribbiani")
session.add(dummyUser)

In [22]:
session.query(Customer).all()

2017-02-17 17:37:44,675 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-17 17:37:44,675 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".lastname AS "Customer_lastname" 
FROM "Customer" 
WHERE "Customer".id = ?
2017-02-17 17:37:44,690 INFO sqlalchemy.engine.base.Engine (1,)
2017-02-17 17:37:44,690 INFO sqlalchemy.engine.base.Engine UPDATE "Customer" SET firstname=? WHERE "Customer".id = ?
2017-02-17 17:37:44,690 INFO sqlalchemy.engine.base.Engine ('Emily', 1)
2017-02-17 17:37:44,706 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:44,706 INFO sqlalchemy.engine.base.Engine ('Joey', 'Tribbiani')
2017-02-17 17:37:44,721 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer"
2017-02-17 17:37:44,721 INFO sqlalchemy.engine.base.Engine ()


[<Customer(firstname='Emily', lastname='Doe')>,
 <Customer(firstname='Abriella', lastname='Peters')>,
 <Customer(firstname='Monica', lastname='Geller')>,
 <Customer(firstname='Chandler', lastname='Bing')>,
 <Customer(firstname='Joey', lastname='Tribbiani')>]

**Note the name change and the record for the dummy user has been inserted.**

In [23]:
session.rollback()

2017-02-17 17:37:44,775 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [24]:
session.query(Customer).all()

2017-02-17 17:37:44,895 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-02-17 17:37:44,895 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer"
2017-02-17 17:37:44,911 INFO sqlalchemy.engine.base.Engine ()


[<Customer(firstname='Jessica', lastname='Doe')>,
 <Customer(firstname='Abriella', lastname='Peters')>,
 <Customer(firstname='Monica', lastname='Geller')>,
 <Customer(firstname='Chandler', lastname='Bing')>]

**All changes have been rolled back.**

<h3>7) Querying</h3><br>
The query() method is used to query the tables. A few examples of querying are given below:

In [25]:
for instance in session.query(Customer).order_by(Customer.id):
     print(instance.firstname, instance.lastname)

2017-02-17 17:37:45,007 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer" ORDER BY "Customer".id
2017-02-17 17:37:45,007 INFO sqlalchemy.engine.base.Engine ()
Jessica Doe
Abriella Peters
Monica Geller
Chandler Bing


In [26]:
for firstname, lastname in session.query(Customer.firstname, Customer.lastname):
     print(firstname, lastname)

2017-02-17 17:37:45,144 INFO sqlalchemy.engine.base.Engine SELECT "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer"
2017-02-17 17:37:45,144 INFO sqlalchemy.engine.base.Engine ()
Jessica Doe
Abriella Peters
Monica Geller
Chandler Bing


In [39]:
for c in session.query(Customer).order_by(Customer.id)[1:3]: #prints 1 and 2 rows
    print(c)

2017-02-17 17:45:21,445 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer" ORDER BY "Customer".id
 LIMIT ? OFFSET ?
2017-02-17 17:45:21,449 INFO sqlalchemy.engine.base.Engine (2, 1)
<Customer(firstname='Abriella', lastname='Peters')>
<Customer(firstname='Monica', lastname='Geller')>


In [28]:
for firstname, in session.query(Customer.firstname).filter_by(lastname = 'Doe'):
    print(firstname)

2017-02-17 17:37:45,420 INFO sqlalchemy.engine.base.Engine SELECT "Customer".firstname AS "Customer_firstname" 
FROM "Customer" 
WHERE "Customer".lastname = ?
2017-02-17 17:37:45,420 INFO sqlalchemy.engine.base.Engine ('Doe',)
Jessica


<h3>8) Building Relationships</h3><br>
Let’s consider how to add a table related to Customer can be mapped and queried. Customers can make orders in the system. This implies a basic one to many association from the Customers to a new table which stores orders, which we will call Orders. Using declarative, we define this table along with its mapped class, Orders:

In [29]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Orders(Base):
    __tablename__ = 'Orders'
    id = Column(Integer, primary_key = True)
    item = Column(String, nullable = False)
    quantity = Column(Integer, nullable = False)
    customerId = Column(Integer, ForeignKey('Customer.id'))

    customer = relationship("Customer", back_populates="orders")

    def __repr__(self):
        return "<Orders(Item = '%s', Quantity = '%s')>" % (self.item, self.quantity)

Customer.orders = relationship("Orders", order_by = Orders.id, back_populates = "customer")

The class Orders has a **foreign key** which indicates that the values in this column must be present as values in the remote column. 

In [30]:
Base.metadata.create_all(engine)

2017-02-17 17:37:45,700 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Customer")
2017-02-17 17:37:45,715 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:45,715 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Orders")
2017-02-17 17:37:45,715 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:45,735 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Orders" (
	id INTEGER NOT NULL, 
	item VARCHAR NOT NULL, 
	quantity INTEGER NOT NULL, 
	"customerId" INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY("customerId") REFERENCES "Customer" (id)
)


2017-02-17 17:37:45,741 INFO sqlalchemy.engine.base.Engine ()
2017-02-17 17:37:45,744 INFO sqlalchemy.engine.base.Engine COMMIT


In [31]:
#let us create a customer object
customer2 = Customer(firstname = "Janice", lastname = "Hosenstein")
#note the orders attribute
customer2.orders

[]

In [32]:
#let's fill a few orders for this customer
customer2.orders = [Orders(item = "Fan", quantity = 2), Orders(item = "Light", quantity = 1)]
customer2.orders

[<Orders(Item = 'Fan', Quantity = '2')>,
 <Orders(Item = 'Light', Quantity = '1')>]

In [33]:
session.add(customer2)
session.query(Orders).all() 

2017-02-17 17:37:46,186 INFO sqlalchemy.engine.base.Engine INSERT INTO "Customer" (firstname, lastname) VALUES (?, ?)
2017-02-17 17:37:46,186 INFO sqlalchemy.engine.base.Engine ('Janice', 'Hosenstein')
2017-02-17 17:37:46,202 INFO sqlalchemy.engine.base.Engine INSERT INTO "Orders" (item, quantity, "customerId") VALUES (?, ?, ?)
2017-02-17 17:37:46,202 INFO sqlalchemy.engine.base.Engine ('Fan', 2, 5)
2017-02-17 17:37:46,220 INFO sqlalchemy.engine.base.Engine INSERT INTO "Orders" (item, quantity, "customerId") VALUES (?, ?, ?)
2017-02-17 17:37:46,224 INFO sqlalchemy.engine.base.Engine ('Light', 1, 5)
2017-02-17 17:37:46,228 INFO sqlalchemy.engine.base.Engine SELECT "Orders".id AS "Orders_id", "Orders".item AS "Orders_item", "Orders".quantity AS "Orders_quantity", "Orders"."customerId" AS "Orders_customerId" 
FROM "Orders"
2017-02-17 17:37:46,233 INFO sqlalchemy.engine.base.Engine ()


[<Orders(Item = 'Fan', Quantity = '2')>,
 <Orders(Item = 'Light', Quantity = '1')>]

As evident from the logging data, entries for the 2 orders have been made in the Orders table as well. This is the fucntion of the **back_populate** attribute in the **relationship()** method.<br><br><br> 

<h3>9) Querying with Joins</h3><br>
For an inner join, query.join() can be used. The join condition will automatically be derived if there is only one foreign key in both the tables. 

In [34]:
session.query(Customer).join(Orders).all()

2017-02-17 17:42:58,945 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer" JOIN "Orders" ON "Customer".id = "Orders"."customerId"
2017-02-17 17:42:58,953 INFO sqlalchemy.engine.base.Engine ()


[<Customer(firstname='Janice', lastname='Hosenstein')>]

In [35]:
session.query(Orders).join(Customer).all()

2017-02-17 17:43:26,349 INFO sqlalchemy.engine.base.Engine SELECT "Orders".id AS "Orders_id", "Orders".item AS "Orders_item", "Orders".quantity AS "Orders_quantity", "Orders"."customerId" AS "Orders_customerId" 
FROM "Orders" JOIN "Customer" ON "Customer".id = "Orders"."customerId"
2017-02-17 17:43:26,357 INFO sqlalchemy.engine.base.Engine ()


[<Orders(Item = 'Fan', Quantity = '2')>,
 <Orders(Item = 'Light', Quantity = '1')>]

In [38]:
for c, o in session.query(Customer, Orders).filter(Customer.id == Orders.customerId).all():
    print(c, o)

2017-02-17 17:44:59,502 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname", "Orders".id AS "Orders_id", "Orders".item AS "Orders_item", "Orders".quantity AS "Orders_quantity", "Orders"."customerId" AS "Orders_customerId" 
FROM "Customer", "Orders" 
WHERE "Customer".id = "Orders"."customerId"
2017-02-17 17:44:59,512 INFO sqlalchemy.engine.base.Engine ()
<Customer(firstname='Janice', lastname='Hosenstein')> <Orders(Item = 'Fan', Quantity = '2')>
<Customer(firstname='Janice', lastname='Hosenstein')> <Orders(Item = 'Light', Quantity = '1')>


<br>However, if there are no foreign keys or there are several foreign keys, we will have to **explicitly mention each join condition.**

In [42]:
session.query(Customer).join(Orders, Customer.id == Orders.customerId).all()

2017-02-17 17:47:59,922 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer" JOIN "Orders" ON "Customer".id = "Orders"."customerId"
2017-02-17 17:47:59,930 INFO sqlalchemy.engine.base.Engine ()


[<Customer(firstname='Janice', lastname='Hosenstein')>]

<br><h3>10)Deleting Records</h3><br>



In [45]:
session.delete(customer1)
session.query(Customer).all()

2017-02-17 17:53:14,298 INFO sqlalchemy.engine.base.Engine SELECT "Orders".id AS "Orders_id", "Orders".item AS "Orders_item", "Orders".quantity AS "Orders_quantity", "Orders"."customerId" AS "Orders_customerId" 
FROM "Orders" 
WHERE ? = "Orders"."customerId" ORDER BY "Orders".id
2017-02-17 17:53:14,306 INFO sqlalchemy.engine.base.Engine (1,)
2017-02-17 17:53:14,314 INFO sqlalchemy.engine.base.Engine DELETE FROM "Customer" WHERE "Customer".id = ?
2017-02-17 17:53:14,318 INFO sqlalchemy.engine.base.Engine (1,)
2017-02-17 17:53:14,322 INFO sqlalchemy.engine.base.Engine SELECT "Customer".id AS "Customer_id", "Customer".firstname AS "Customer_firstname", "Customer".lastname AS "Customer_lastname" 
FROM "Customer"
2017-02-17 17:53:14,326 INFO sqlalchemy.engine.base.Engine ()


[<Customer(firstname='Abriella', lastname='Peters')>,
 <Customer(firstname='Monica', lastname='Geller')>,
 <Customer(firstname='Chandler', lastname='Bing')>,
 <Customer(firstname='Janice', lastname='Hosenstein')>]